In [1]:
import os
import wget
import zipfile
import numpy as np
import pandas as pd
import datetime

In [38]:
def get_tick_b3(date):
    url = 'https://arquivos.b3.com.br/apinegocios/tickercsv/'+ date
    file_name = str(date+'_B3_TickData.zip')
    wget.download(url,file_name)
    
    # Get the directory of the zip file
    zip_dir = os.path.dirname(file_name)
    
    # Create a ZipFile object
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        # Extract all the contents into the directory of the zip file
        zip_ref.extractall(zip_dir)
    print('/n')
    print("1/8 - Extracted all contents of ",file_name)
    
    # Read Extracted File
    folder_ref = os.getcwd()
    files = os.listdir(folder_ref)
    files_txt = [i for i in files if i.endswith('_NEGOCIOSAVISTA.txt')]
        
    df = pd.read_csv(files_txt[0], sep=";")

    # Update PrecoNegocio
    df['PrecoNegocio'] = df.PrecoNegocio.str.replace(",", ".").astype('float')
    print('2/8 - PrecoNegocio Updated')

    # Update Codigos Participantes
    df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']] = df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']].fillna(0)
    df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']] = df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']].astype('int').astype('str')
    print('3/8 - Codigos Participantes Updated')

    #Update Datetime
    # Ensure 'HoraFechamento' is a string and pad with leading zeros to ensure it's 9 characters long
    df['HoraFechamento'] = df['HoraFechamento'].astype(str).str.zfill(9)
    # Extract the hours, minutes, seconds, and milliseconds
    df['HoraFechamento'] = df['HoraFechamento'].apply(
        lambda x: f"{x[:2]}:{x[2:4]}:{x[4:6]}.{x[6:9]}"
    )
    # Ensure 'HoraFechamento' is a string
    df['HoraFechamento'] = df['HoraFechamento'].astype(str)
    # Convert the 'HoraFechamento' to datetime with the appropriate format including milliseconds
    df['HoraFechamento'] = pd.to_datetime(df['HoraFechamento'], format='%H:%M:%S.%f').dt.time
    print('4/8 - HoraFechamento Updated')

    #Create a New Index
    str1 = df.CodigoInstrumento
    str2 = df.CodigoIdentificadorNegocio.astype(str)
    str3 = df.DataReferencia.astype(str)
    str4 = df.HoraFechamento.astype(str)
    newindex = str1+'_'+str2+'_'+str3+'_'+str4
    df['Index'] = newindex
    # Set 'HoraFechamento' column as the index
    df = df.set_index('Index')
    print('5/8 - New_Index Created')

    # Remove a column inplace
    df.drop(columns=['AcaoAtualizacao','TipoSessaoPregao','DataNegocio'], inplace=True)
    print('6/8 - Columns Remove Updated')
    #Rename Columns
    dicionario = {'DataReferencia' : 'Dia', 'CodigoInstrumento' : 'Instrumento', 'PrecoNegocio' : 'Preco', 'QuantidadeNegociada' : 'Quantidade', 'HoraFechamento' : 'Hora', 'CodigoIdentificadorNegocio' : 'Cod_Negocio', 'CodigoParticipanteComprador' : 'Comprador', 'CodigoParticipanteVendedor' : 'Vendedor'}
    df.rename(dicionario, axis = 1, inplace=True)
    print('7/8 - Columns Rename Updated')
    #Reorder Columns
    new_order = ['Cod_Negocio', 'Instrumento', 'Dia', 'Hora','Preco','Quantidade','Comprador','Vendedor']
    df = df[new_order]
    print('(8/8 - Columns New Order Updated')
    print('Data Extraction and Transformation - Done')
    
    return df

In [39]:
df = get_tick_b3('2024-07-15')

100% [........................................................................] 70189868 / 70189868/n
1/8 - Extracted all contents of  2024-07-15_B3_TickData.zip
2/8 - PrecoNegocio Updated
3/8 - Codigos Participantes Updated
4/8 - HoraFechamento Updated
5/8 - New_Index Created
6/8 - Columns Remove Updated
7/8 - Columns Rename Updated
(8/8 - Columns New Order Updated
Data Extraction and Transformation - Done


In [4]:
df

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
Index,,,,,,,,,,,
TF583R_10_2024-07-15_03:08:19.617000,2024-07-15,TF583R,0,10.00,10000,03:08:19.617000,10,1,2024-07-15,100,100
CCMN24_120_2024-07-15_09:00:04.270000,2024-07-15,CCMN24,0,56.55,1,09:00:04.270000,120,1,2024-07-15,0,0
CCMN24_130_2024-07-15_09:00:07.950000,2024-07-15,CCMN24,0,56.51,1,09:00:07.950000,130,1,2024-07-15,0,0
CCMN24_140_2024-07-15_09:00:12.433000,2024-07-15,CCMN24,0,56.51,1,09:00:12.433000,140,1,2024-07-15,0,0
DI1F31_10_2024-07-15_09:00:12.890000,2024-07-15,DI1F31,0,11.80,1,09:00:12.890000,10,1,2024-07-15,88,120
...,...,...,...,...,...,...,...,...,...,...,...
INDQ24_98250_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98250,1,2024-07-15,93,16
INDQ24_98260_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98260,1,2024-07-15,93,16
INDQ24_98270_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98270,1,2024-07-15,120,16


In [74]:
df[df['CodigoInstrumento']=='PETR4']

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
Index,,,,,,,,,,,
PETR4_10_2024-07-15_10:03:47.900000,2024-07-15,PETR4,0,38.30,1400,10:03:47.900000,10,1,2024-07-15,1618,120
PETR4_20_2024-07-15_10:03:47.900000,2024-07-15,PETR4,0,38.30,1400,10:03:47.900000,20,1,2024-07-15,1618,23
PETR4_30_2024-07-15_10:03:47.900000,2024-07-15,PETR4,0,38.30,1000,10:03:47.900000,30,1,2024-07-15,1618,3
PETR4_40_2024-07-15_10:03:47.900000,2024-07-15,PETR4,0,38.30,100,10:03:47.900000,40,1,2024-07-15,1618,4090
PETR4_50_2024-07-15_10:03:47.900000,2024-07-15,PETR4,0,38.30,100,10:03:47.900000,50,1,2024-07-15,1618,120
...,...,...,...,...,...,...,...,...,...,...,...
PETR4_511750_2024-07-15_17:59:58.953000,2024-07-15,PETR4,0,38.46,100,17:59:58.953000,511750,6,2024-07-15,735,85
PETR4_511760_2024-07-15_17:59:58.953000,2024-07-15,PETR4,0,38.46,400,17:59:58.953000,511760,6,2024-07-15,3,85
PETR4_511770_2024-07-15_17:59:58.967000,2024-07-15,PETR4,0,38.46,500,17:59:58.967000,511770,6,2024-07-15,3,85


### Transformação 1: "PrecoNegocio" com divisor decimal como "vírgula"

In [6]:
df['PrecoNegocio'] = df.PrecoNegocio.str.replace(",", ".").astype('float')

In [7]:
df

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-07-15,TF583R,0,10.00,10000,30819617,10,1,2024-07-15,100.0,100.0
1,2024-07-15,CCMN24,0,56.55,1,90004270,120,1,2024-07-15,NaN,NaN
2,2024-07-15,CCMN24,0,56.51,1,90007950,130,1,2024-07-15,NaN,NaN
3,2024-07-15,CCMN24,0,56.51,1,90012433,140,1,2024-07-15,NaN,NaN
4,2024-07-15,DI1F31,0,11.80,1,90012890,10,1,2024-07-15,88.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...
8260063,2024-07-15,INDQ24,0,130530.00,5,183122113,98250,1,2024-07-15,93.0,16.0
8260064,2024-07-15,INDQ24,0,130530.00,5,183122113,98260,1,2024-07-15,93.0,16.0
8260065,2024-07-15,INDQ24,0,130530.00,5,183122113,98270,1,2024-07-15,120.0,16.0
8260066,2024-07-15,INDQ24,0,130530.00,10,183122113,98280,1,2024-07-15,3.0,16.0


### Transformacao 2 Substituir NaN nas colunas "CodigoParticipanteComprador" e "CodigoParticipanteVendedor" para 0;
### Colunas "CodigoParticipanteComprador" e "CodigoParticipanteVendedor" para int (remover casas decimais) e depois para str;

In [11]:
    df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']] = df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']].fillna(0)
    df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']] = df[['CodigoParticipanteComprador','CodigoParticipanteVendedor']].astype('int').astype('str')

In [12]:
df

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-07-15,TF583R,0,10.00,10000,30819617,10,1,2024-07-15,100,100
1,2024-07-15,CCMN24,0,56.55,1,90004270,120,1,2024-07-15,0,0
2,2024-07-15,CCMN24,0,56.51,1,90007950,130,1,2024-07-15,0,0
3,2024-07-15,CCMN24,0,56.51,1,90012433,140,1,2024-07-15,0,0
4,2024-07-15,DI1F31,0,11.80,1,90012890,10,1,2024-07-15,88,120
...,...,...,...,...,...,...,...,...,...,...,...
8260063,2024-07-15,INDQ24,0,130530.00,5,183122113,98250,1,2024-07-15,93,16
8260064,2024-07-15,INDQ24,0,130530.00,5,183122113,98260,1,2024-07-15,93,16
8260065,2024-07-15,INDQ24,0,130530.00,5,183122113,98270,1,2024-07-15,120,16
8260066,2024-07-15,INDQ24,0,130530.00,10,183122113,98280,1,2024-07-15,3,16


### Transformação 3 - Data em miliseconds

In [24]:
# Ensure 'HoraFechamento' is a string and pad with leading zeros to ensure it's 9 characters long
df['HoraFechamento'] = df['HoraFechamento'].astype(str).str.zfill(9)

In [25]:
df

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-07-15,TF583R,0,10.00,10000,030819617,10,1,2024-07-15,100,100
1,2024-07-15,CCMN24,0,56.55,1,090004270,120,1,2024-07-15,0,0
2,2024-07-15,CCMN24,0,56.51,1,090007950,130,1,2024-07-15,0,0
3,2024-07-15,CCMN24,0,56.51,1,090012433,140,1,2024-07-15,0,0
4,2024-07-15,DI1F31,0,11.80,1,090012890,10,1,2024-07-15,88,120
...,...,...,...,...,...,...,...,...,...,...,...
8260063,2024-07-15,INDQ24,0,130530.00,5,183122113,98250,1,2024-07-15,93,16
8260064,2024-07-15,INDQ24,0,130530.00,5,183122113,98260,1,2024-07-15,93,16
8260065,2024-07-15,INDQ24,0,130530.00,5,183122113,98270,1,2024-07-15,120,16
8260066,2024-07-15,INDQ24,0,130530.00,10,183122113,98280,1,2024-07-15,3,16


In [27]:
    # Extract the hours, minutes, seconds, and milliseconds
    df['HoraFechamento'] = df['HoraFechamento'].apply(
        lambda x: f"{x[:2]}:{x[2:4]}:{x[4:6]}.{x[6:9]}"
    )

In [28]:
df

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-07-15,TF583R,0,10.00,10000,03:08:19.617,10,1,2024-07-15,100,100
1,2024-07-15,CCMN24,0,56.55,1,09:00:04.270,120,1,2024-07-15,0,0
2,2024-07-15,CCMN24,0,56.51,1,09:00:07.950,130,1,2024-07-15,0,0
3,2024-07-15,CCMN24,0,56.51,1,09:00:12.433,140,1,2024-07-15,0,0
4,2024-07-15,DI1F31,0,11.80,1,09:00:12.890,10,1,2024-07-15,88,120
...,...,...,...,...,...,...,...,...,...,...,...
8260063,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113,98250,1,2024-07-15,93,16
8260064,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113,98260,1,2024-07-15,93,16
8260065,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113,98270,1,2024-07-15,120,16
8260066,2024-07-15,INDQ24,0,130530.00,10,18:31:22.113,98280,1,2024-07-15,3,16


In [32]:
type(df['HoraFechamento'][0])

str

In [ ]:
# Convert the 'HoraFechamento' to datetime with the appropriate format including milliseconds
df['HoraFechamento'] = pd.to_datetime(df['HoraFechamento'], format='%H%M%S.%f').dt.time

In [34]:
df

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
0,2024-07-15,TF583R,0,10.00,10000,03:08:19.617000,10,1,2024-07-15,100,100
1,2024-07-15,CCMN24,0,56.55,1,09:00:04.270000,120,1,2024-07-15,0,0
2,2024-07-15,CCMN24,0,56.51,1,09:00:07.950000,130,1,2024-07-15,0,0
3,2024-07-15,CCMN24,0,56.51,1,09:00:12.433000,140,1,2024-07-15,0,0
4,2024-07-15,DI1F31,0,11.80,1,09:00:12.890000,10,1,2024-07-15,88,120
...,...,...,...,...,...,...,...,...,...,...,...
8260063,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98250,1,2024-07-15,93,16
8260064,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98260,1,2024-07-15,93,16
8260065,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98270,1,2024-07-15,120,16
8260066,2024-07-15,INDQ24,0,130530.00,10,18:31:22.113000,98280,1,2024-07-15,3,16


In [48]:
df.dtypes

DataReferencia                  object
CodigoInstrumento               object
AcaoAtualizacao                  int64
PrecoNegocio                   float64
QuantidadeNegociada              int64
HoraFechamento                  object
CodigoIdentificadorNegocio       int64
TipoSessaoPregao                 int64
DataNegocio                     object
CodigoParticipanteComprador     object
CodigoParticipanteVendedor      object
dtype: object

### 4 Transformacao Criar um Indice;
CodigoInstrumento + CodigoIdentificadorNegocio + DataReferencia + HoraFechamento

In [54]:
str1 = df.CodigoInstrumento

In [55]:
str1

0          TF583R
1          CCMN24
2          CCMN24
3          CCMN24
4          DI1F31
            ...  
8260063    INDQ24
8260064    INDQ24
8260065    INDQ24
8260066    INDQ24
8260067    INDQ24
Name: CodigoInstrumento, Length: 8260068, dtype: object

In [51]:
type(str1)

str

In [52]:
str2 = df.CodigoIdentificadorNegocio.astype(str)

In [53]:
str2

0             10
1            120
2            130
3            140
4             10
           ...  
8260063    98250
8260064    98260
8260065    98270
8260066    98280
8260067    98290
Name: CodigoIdentificadorNegocio, Length: 8260068, dtype: object

In [56]:
str3 = df.DataReferencia.astype(str)

In [57]:
str3

0          2024-07-15
1          2024-07-15
2          2024-07-15
3          2024-07-15
4          2024-07-15
              ...    
8260063    2024-07-15
8260064    2024-07-15
8260065    2024-07-15
8260066    2024-07-15
8260067    2024-07-15
Name: DataReferencia, Length: 8260068, dtype: object

In [58]:
str4 = df.HoraFechamento.astype(str)

In [59]:
str4

0          03:08:19.617000
1          09:00:04.270000
2          09:00:07.950000
3          09:00:12.433000
4          09:00:12.890000
                ...       
8260063    18:31:22.113000
8260064    18:31:22.113000
8260065    18:31:22.113000
8260066    18:31:22.113000
8260067    18:31:22.113000
Name: HoraFechamento, Length: 8260068, dtype: object

In [60]:
newindex = str1+'_'+str2+'_'+str3+'_'+str4

In [61]:
newindex

0             TF583R_10_2024-07-15_03:08:19.617000
1            CCMN24_120_2024-07-15_09:00:04.270000
2            CCMN24_130_2024-07-15_09:00:07.950000
3            CCMN24_140_2024-07-15_09:00:12.433000
4             DI1F31_10_2024-07-15_09:00:12.890000
                            ...                   
8260063    INDQ24_98250_2024-07-15_18:31:22.113000
8260064    INDQ24_98260_2024-07-15_18:31:22.113000
8260065    INDQ24_98270_2024-07-15_18:31:22.113000
8260066    INDQ24_98280_2024-07-15_18:31:22.113000
8260067    INDQ24_98290_2024-07-15_18:31:22.113000
Length: 8260068, dtype: object

In [62]:
duplicates = newindex.duplicated()

In [65]:
duplicates.value_counts()

False    8260068
Name: count, dtype: int64

In [66]:
df['Index'] = newindex

In [67]:
df

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor,Index
0,2024-07-15,TF583R,0,10.00,10000,03:08:19.617000,10,1,2024-07-15,100,100,TF583R_10_2024-07-15_03:08:19.617000
1,2024-07-15,CCMN24,0,56.55,1,09:00:04.270000,120,1,2024-07-15,0,0,CCMN24_120_2024-07-15_09:00:04.270000
2,2024-07-15,CCMN24,0,56.51,1,09:00:07.950000,130,1,2024-07-15,0,0,CCMN24_130_2024-07-15_09:00:07.950000
3,2024-07-15,CCMN24,0,56.51,1,09:00:12.433000,140,1,2024-07-15,0,0,CCMN24_140_2024-07-15_09:00:12.433000
4,2024-07-15,DI1F31,0,11.80,1,09:00:12.890000,10,1,2024-07-15,88,120,DI1F31_10_2024-07-15_09:00:12.890000
...,...,...,...,...,...,...,...,...,...,...,...,...
8260063,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98250,1,2024-07-15,93,16,INDQ24_98250_2024-07-15_18:31:22.113000
8260064,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98260,1,2024-07-15,93,16,INDQ24_98260_2024-07-15_18:31:22.113000
8260065,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98270,1,2024-07-15,120,16,INDQ24_98270_2024-07-15_18:31:22.113000
8260066,2024-07-15,INDQ24,0,130530.00,10,18:31:22.113000,98280,1,2024-07-15,3,16,INDQ24_98280_2024-07-15_18:31:22.113000


In [68]:
# Set 'HoraFechamento' column as the index
df = df.set_index('Index')

In [69]:
df

,DataReferencia,CodigoInstrumento,AcaoAtualizacao,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,TipoSessaoPregao,DataNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
Index,,,,,,,,,,,
TF583R_10_2024-07-15_03:08:19.617000,2024-07-15,TF583R,0,10.00,10000,03:08:19.617000,10,1,2024-07-15,100,100
CCMN24_120_2024-07-15_09:00:04.270000,2024-07-15,CCMN24,0,56.55,1,09:00:04.270000,120,1,2024-07-15,0,0
CCMN24_130_2024-07-15_09:00:07.950000,2024-07-15,CCMN24,0,56.51,1,09:00:07.950000,130,1,2024-07-15,0,0
CCMN24_140_2024-07-15_09:00:12.433000,2024-07-15,CCMN24,0,56.51,1,09:00:12.433000,140,1,2024-07-15,0,0
DI1F31_10_2024-07-15_09:00:12.890000,2024-07-15,DI1F31,0,11.80,1,09:00:12.890000,10,1,2024-07-15,88,120
...,...,...,...,...,...,...,...,...,...,...,...
INDQ24_98250_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98250,1,2024-07-15,93,16
INDQ24_98260_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98260,1,2024-07-15,93,16
INDQ24_98270_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,0,130530.00,5,18:31:22.113000,98270,1,2024-07-15,120,16


### Transformacao 5 - Remover Renomear Reorganizar colunas

#### 5.1 Remover coluna AcaoAtualizacao

In [76]:
df.AcaoAtualizacao.value_counts()

AcaoAtualizacao
0    8260064
2          4
Name: count, dtype: int64

#### 5.2 Remover AcaoAtualizacao

In [77]:
df.TipoSessaoPregao.value_counts()

TipoSessaoPregao
1    8243365
6      16703
Name: count, dtype: int64

In [19]:
# Remove a column inplace
df.drop(columns=['AcaoAtualizacao','TipoSessaoPregao','DataNegocio'], inplace=True)

In [20]:
df

,DataReferencia,CodigoInstrumento,PrecoNegocio,QuantidadeNegociada,HoraFechamento,CodigoIdentificadorNegocio,CodigoParticipanteComprador,CodigoParticipanteVendedor
Index,,,,,,,,
TF583R_10_2024-07-15_03:08:19.617000,2024-07-15,TF583R,10.00,10000,03:08:19.617000,10,100,100
CCMN24_120_2024-07-15_09:00:04.270000,2024-07-15,CCMN24,56.55,1,09:00:04.270000,120,0,0
CCMN24_130_2024-07-15_09:00:07.950000,2024-07-15,CCMN24,56.51,1,09:00:07.950000,130,0,0
CCMN24_140_2024-07-15_09:00:12.433000,2024-07-15,CCMN24,56.51,1,09:00:12.433000,140,0,0
DI1F31_10_2024-07-15_09:00:12.890000,2024-07-15,DI1F31,11.80,1,09:00:12.890000,10,88,120
...,...,...,...,...,...,...,...,...
INDQ24_98250_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,130530.00,5,18:31:22.113000,98250,93,16
INDQ24_98260_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,130530.00,5,18:31:22.113000,98260,93,16
INDQ24_98270_2024-07-15_18:31:22.113000,2024-07-15,INDQ24,130530.00,5,18:31:22.113000,98270,120,16


In [21]:
df.columns

Index(['DataReferencia', 'CodigoInstrumento', 'PrecoNegocio',
       'QuantidadeNegociada', 'HoraFechamento', 'CodigoIdentificadorNegocio',
       'CodigoParticipanteComprador', 'CodigoParticipanteVendedor'],
      dtype='object')

### Renomear colunas

Usamos um dicionário para renomear as colunas, contendo a estrutura {'nome antigo' : 'novo nome'}

In [28]:
dicionario = {'DataReferencia' : 'Dia', 'CodigoInstrumento' : 'Instrumento', 'PrecoNegocio' : 'Preco', 'QuantidadeNegociada' : 'Quantidade', 'HoraFechamento' : 'Hora', 'CodigoIdentificadorNegocio' : 'Cod_Negocio', 'CodigoParticipanteComprador' : 'Comprador', 'CodigoParticipanteVendedor' : 'Vendedor'}

In [29]:
df.rename(dicionario, axis = 1, inplace=True)

### Reordenar colunas

In [31]:
new_order = ['Cod_Negocio', 'Instrumento', 'Dia', 'Hora','Preco','Quantidade','Comprador','Vendedor']

In [32]:
# Reordering columns using reindex
df = df[new_order]

In [30]:
df.columns

Index(['Dia', 'Instrumento', 'Preco', 'Quantidade', 'Hora', 'Cod_Negocio',
       'Comprador', 'Vendedor'],
      dtype='object')

In [33]:
df

,Cod_Negocio,Instrumento,Dia,Hora,Preco,Quantidade,Comprador,Vendedor
Index,,,,,,,,
TF583R_10_2024-07-15_03:08:19.617000,10,TF583R,2024-07-15,03:08:19.617000,10.00,10000,100,100
CCMN24_120_2024-07-15_09:00:04.270000,120,CCMN24,2024-07-15,09:00:04.270000,56.55,1,0,0
CCMN24_130_2024-07-15_09:00:07.950000,130,CCMN24,2024-07-15,09:00:07.950000,56.51,1,0,0
CCMN24_140_2024-07-15_09:00:12.433000,140,CCMN24,2024-07-15,09:00:12.433000,56.51,1,0,0
DI1F31_10_2024-07-15_09:00:12.890000,10,DI1F31,2024-07-15,09:00:12.890000,11.80,1,88,120
...,...,...,...,...,...,...,...,...
INDQ24_98250_2024-07-15_18:31:22.113000,98250,INDQ24,2024-07-15,18:31:22.113000,130530.00,5,93,16
INDQ24_98260_2024-07-15_18:31:22.113000,98260,INDQ24,2024-07-15,18:31:22.113000,130530.00,5,93,16
INDQ24_98270_2024-07-15_18:31:22.113000,98270,INDQ24,2024-07-15,18:31:22.113000,130530.00,5,120,16


In [40]:
df

,Cod_Negocio,Instrumento,Dia,Hora,Preco,Quantidade,Comprador,Vendedor
Index,,,,,,,,
TF583R_10_2024-07-15_03:08:19.617000,10,TF583R,2024-07-15,03:08:19.617000,10.00,10000,100,100
CCMN24_120_2024-07-15_09:00:04.270000,120,CCMN24,2024-07-15,09:00:04.270000,56.55,1,0,0
CCMN24_130_2024-07-15_09:00:07.950000,130,CCMN24,2024-07-15,09:00:07.950000,56.51,1,0,0
CCMN24_140_2024-07-15_09:00:12.433000,140,CCMN24,2024-07-15,09:00:12.433000,56.51,1,0,0
DI1F31_10_2024-07-15_09:00:12.890000,10,DI1F31,2024-07-15,09:00:12.890000,11.80,1,88,120
...,...,...,...,...,...,...,...,...
INDQ24_98250_2024-07-15_18:31:22.113000,98250,INDQ24,2024-07-15,18:31:22.113000,130530.00,5,93,16
INDQ24_98260_2024-07-15_18:31:22.113000,98260,INDQ24,2024-07-15,18:31:22.113000,130530.00,5,93,16
INDQ24_98270_2024-07-15_18:31:22.113000,98270,INDQ24,2024-07-15,18:31:22.113000,130530.00,5,120,16
